# Resume Classifier Project





## Imports

In [6]:
# !pip install PyPDF2
# !pip install gradio-pdf
# !pip install fitz
# !pip install pymupdf
# !pip install nltk
import tensorflow as tf, keras, numpy as np, pandas as pd
# import kagglehub
import nltk, os
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
import PyPDF2
from gradio_pdf import PDF
import re
import joblib
import os
import fitz

## Dataset

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

# files.upload()

# !ls /content/drive/MyDrive/ResumeClassifier

# drive_path = '/content/drive/MyDrive/ResumeClassifier/resume-dataset'
# path = kagglehub.dataset_download("gauravduttakiit/resume-dataset", path=drive_path)
# !kaggle datasets download -d gauravduttakiit/resume-dataset
# !unzip dataset-name.zip -d /content/drive/MyDrive/ResumeClassifier

# file_path = '/content/drive/MyDrive/ResumeClassifier/UpdatedResumeDataSet.csv'

In [8]:
file_path = 'https://raw.githubusercontent.com/BrandonYChan/Resume-Screener/refs/heads/main/UpdatedResumeDataSet.csv'
resume_df = pd.read_csv(file_path)
resume_df['Category'].unique()

array(['Data Science', 'HR', 'Advocate', 'Arts', 'Web Designing',
       'Mechanical Engineer', 'Sales', 'Health and fitness',
       'Civil Engineer', 'Java Developer', 'Business Analyst',
       'SAP Developer', 'Automation Testing', 'Electrical Engineering',
       'Operations Manager', 'Python Developer', 'DevOps Engineer',
       'Network Security Engineer', 'PMO', 'Database', 'Hadoop',
       'ETL Developer', 'DotNet Developer', 'Blockchain', 'Testing'],
      dtype=object)

In [9]:
resume_df

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...
958,Testing,â Willingness to accept the challenges. â ...
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [10]:
# Clean text
resume_df['Resume'] = resume_df['Resume'].str.lower()
resume_df['Resume'] = resume_df['Resume'].str.replace(r'[^a-zA-z\s]', '', regex=True)
resume_df['Resume'] = resume_df['Resume'].str.strip().replace(r'\s', ' ', regex=True)

# Remove stopwords
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
resume_df['Resume'] = resume_df['Resume'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

resume_df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bchan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Category,Resume
0,Data Science,skills programming languages python pandas num...
1,Data Science,education details may may uitrgpv data scienti...
2,Data Science,areas interest deep learning control system de...
3,Data Science,skills r python sap hana tableau sap hana sql ...
4,Data Science,education details mca ymcaust faridabad haryan...
...,...,...
957,Testing,computer skills proficient ms office word basi...
958,Testing,willingness accept challenges positive thinkin...
959,Testing,personal skills quick learner eagerness learn ...
960,Testing,computer skills software knowledge mspower poi...


##Model (initial idea)
####Step 1
feed cleaned input resume into a model (such as a neural network) trained on the dataset above, which classifies resumes into a category (e.g., Data Science).
####Step 2
If input resume matches the user-specified category, compare the resume to the job description by searching for key words/skills. If the similarity (e.g., cosine similarity) between the resume and job description is above a certain threshold, assign the resume as 'passable'. In all other cases, assign the resume as 'not passable'.
####Extra
Maybe we can allow the user to feed in a batch of submitted resumes for a job description, where the code would return a small subset of resumes accepted for interviews. These accepted resumes must have a matching category (assigned by the NN model) to the job description, and must also be the top-performers in terms of cosine similarity to the job description.

In [11]:
# Additional imports

# !pip install PyPDF2
# !pip install gradio-pdf

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
import PyPDF2
from gradio_pdf import PDF
import re
import joblib
import os

In [12]:
# Encode labels
resume_df_encoded = resume_df.copy()
label_encoder = LabelEncoder()
resume_df_encoded['Category'] = label_encoder.fit_transform(resume_df_encoded['Category'])
Y = to_categorical(resume_df_encoded['Category'])
Y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [13]:
# Extract features from text in Resume column
tfidf = TfidfVectorizer(max_features=5000) # limit number of features to the top 5000 words with the highest TF-IDF scores across the dataset
X = tfidf.fit_transform(resume_df_encoded['Resume']).toarray() # assign weights to words based on their frequency in a resume and rarity across all resumes
X # each row represents a resume, and each column represets a word's TF-IDF score; there are 5000 columns, each representing one of the top 5000 most important words

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
# Split data into 70% training, 15% val, 15% test
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.3, random_state=13)
X_val, X_test, Y_val, Y_test = train_test_split(X_val, Y_val, test_size=0.5, random_state=13)
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape

((673, 5000), (673, 25), (144, 5000), (144, 25), (145, 5000), (145, 25))

In [15]:
model_path = '/model.pkl'

# Train model if it hasn't already been trained and saved
if not(os.path.exists(model_path)):
  # Feed data into NN
  model = Sequential([
      Input(shape=(X.shape[1],)),
      Dense(512, activation='relu'),
      Dropout(0.3),
      Dense(256, activation='relu'),
      Dropout(0.3),
      Dense(128, activation='relu'),
      Dropout(0.3),
      Dense(Y.shape[1], activation='softmax')
  ])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=5, batch_size=16, verbose=True)
  joblib.dump(model, 'model.pkl')
else:
  model = joblib.load('model.pkl')

test_loss, test_accuracy = model.evaluate(X_test, Y_test)
print(f"Test Accuracy: {test_accuracy}, Test Loss: {test_loss}")

Epoch 1/5
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.0918 - loss: 3.1344 - val_accuracy: 0.4097 - val_loss: 2.2468
Epoch 2/5
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6151 - loss: 1.7173 - val_accuracy: 0.9861 - val_loss: 0.2846
Epoch 3/5
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9591 - loss: 0.3035 - val_accuracy: 0.9931 - val_loss: 0.0456
Epoch 4/5
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9948 - loss: 0.0720 - val_accuracy: 0.9931 - val_loss: 0.0285
Epoch 5/5
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 1.0000 - loss: 0.0304 - val_accuracy: 0.9931 - val_loss: 0.0190
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9941 - loss: 0.0568  
Test Accuracy: 0.9862068891525269, Test Loss: 0.08078958094120026


In [16]:
# Show prediction results of some random samples
random_indices = np.random.choice(X.shape[0], size=5, replace=False) # SRSWOR of 5 random row indices
sample_X = X[random_indices]
sample_Y = Y[random_indices]
predictions = model.predict(sample_X)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))
true_labels = label_encoder.inverse_transform(np.argmax(sample_Y, axis=1))
for i in range(5):
    print(f"Random Resume {i+1}:\n--Predicted = {predicted_labels[i]}\n--True = {true_labels[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Random Resume 1:
--Predicted = Python Developer
--True = Python Developer
Random Resume 2:
--Predicted = ETL Developer
--True = ETL Developer
Random Resume 3:
--Predicted = Mechanical Engineer
--True = Mechanical Engineer
Random Resume 4:
--Predicted = Blockchain
--True = Blockchain
Random Resume 5:
--Predicted = Data Science
--True = Data Science


## Testing Input

In [17]:
# # Import PDF of some random resume
# print("Select a resume to categorize:")
# upload = files.upload()
# file_path = list(upload.keys())[0]
# reader = PyPDF2.PdfReader(file_path)

In [18]:
# # nltk.download('stopwords')
# # stop_words = set(nltk.corpus.stopwords.words('english'))

# def evaluate_pdf(reader):
#   # Read resume text
#   resume_text = ''
#   for page in range(len(reader.pages)):
#       text = reader.pages[page].extract_text()
#       resume_text += text
#   # print("Resume text (original):\n", resume_text)

#   # Clean text (done in the same way as above in the Dataset section)
#   resume_text = resume_text.lower()
#   resume_text = re.sub(r'[^a-zA-z\s]', '', resume_text)
#   resume_text = re.sub(r'\s', ' ', resume_text).strip()

#   resume_text = ' '.join([word for word in resume_text.split() if word not in stop_words])
#   # print("\nResume text (cleaned):\n", resume_text)

#   # Categorize resume using the model
#   resume_text = tfidf.transform([resume_text]).toarray()
#   prediction = model.predict(resume_text)
#   predicted_label = label_encoder.inverse_transform(np.argmax(prediction, axis=1))
#   # print(f"\nImported Resume:\n--Predicted = {predicted_label}")
#   return predicted_label[0]
# evaluate_pdf(reader)


## Gradio App

In [ ]:
    def evaluate_resume(resume_file_path):
        resume_text = ""
        with fitz.open(resume_file_path) as resume:
            if resume.page_count == 0:
                return "PDF has no pages or could not be opened."

            for page_num in range(resume.page_count):
                page = resume[page_num]
                resume_text += page.get_text("text")  # Extract text from each page
        resume_text = resume_text.lower()
        resume_text = re.sub(r'[^a-zA-z\s]', '', resume_text)
        resume_text = re.sub(r'\s', ' ', resume_text).strip()
        resume_text = ' '.join([word for word in resume_text.split() if word not in stop_words])

        # Categorize resume using the model
        resume_text = tfidf.transform([resume_text]).toarray()
        prediction = model.predict(resume_text)
        predicted_label = label_encoder.inverse_transform(np.argmax(prediction, axis=1))

        # Send data to the Django backend
        url = "http://127.0.0.1:7860/save-input/"  # Adjust based on your Django server URL
        data = {'name': name}
        try:
        response = requests.post(url, json=data)
        if response.status_code == 200:
        return f"Hello, {name}! (Saved to database)"
        else:
        return f"Error: {response.json().get('error', 'Unknown error')}"
        except requests.exceptions.RequestException as e:
        return f"Server error: {e}"

        return predicted_label[0]

In [20]:
import gradio
demo = gradio.Interface(
    fn=evaluate_resume,
    inputs=[PDF(label="Document")],
    outputs= [gradio.Textbox(label="Classification Prediction")],
    title="Resume Classifier"
)
demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Keyboard interruption in main thread... closing server.
